# Discrete Model.

In [1]:
from scipy.optimize import minimize
import pandas as pd
import numpy as np
from pyomo.environ import * 
import pandas
import math

# Warehouse Location Problem.

### Problem Statement


This problem is modeled after Amazon who need to determine what is the optimal location to build a warehouse, when taking into account certain customer locations it needs to tranport to. Where we need to take into account the demand of each customer. With this in mind, the objective function would be to minimize cost of transportation using the cost of transporting a good from a possible warehouse i to customer location j. This would mean that if the demand is higher and more goods need to be transported, then the cost would also increase.

We need to determine the optimal locations for the warehouses in order to minimize transportations costs to customers. This is done with the following variables.

#### Decision Variables:

- $y_{i}$ - Binary Variable that indicates whether to build a warehouse at location i.

$$y_{i}=\left\{\begin{array}{ll}
1 & \textrm{if location i is chosen to build a warehouse}\\
0 & \textrm{otherwise}
\end{array}\right.$$


#### Parameters:

- $c_{ij}$: Cost of transporting goods from warehouse at location i to customer j.
- $d_{j}$: Demand of customer j.
- $s_{i}$: Total stock of each warehouse (i).


In this case instead of using the traditional model.I and model.J we use, model.Warehouse and model.Customers. In this case the i and j subscript are done as the following.

- Customers (C): Customers that require merchandise to be delivered -> j subscript.
- Warehouse (W): Warehouse location to be built (if it is optimal) -> i subscript.

With the following indices:

- $i$: Warehouse location. $i \in [1,5] \in W$ (There are 5 warehouse locations - **Madrid, Barcelona, Malaga, Coruna, or Sevilla**.)
- $j$: Customer number. $j \in [1,7] \in C$ (There are 7 customer locations - **Burgos, Zaragoza, Albacete, Murcia, Cordoba, Salamance, Gijon**.) 



This can be seen in the following picture (Possible warehouse locations in red, possible customer locations in blue):

!['map_warehouse'](img/map_warehouse.png)




#### Constraints:

1. Stock of the warehouse must be greater than or equal to the demand of all the customers it will transport to.

$$\sum_{i\in W}  y_{i} \cdot d_{j} \leq s_{i} \qquad (\text{for j} \in \text{C}, (\text{i} \in \text{W}))$$

2. Demand for each warehouse to customer must be met:

$$\sum^{7}_{i=1} y_{i} \cdot d_{j} \geq d_{j} \quad (\text{for j} \in \text{C})$$ 

3. Make sure to have two solutions (two locations to be built):

$$\sum_{i \in W} y_{i} = 2$$


#### Objective Function:

$$\text{minimize} \sum_{i \in W, j \in C} c_{ij}\cdot y_{i}\cdot d_{j}$$




In [8]:
from pyomo.environ import *

# Create Pyomo model
model = ConcreteModel()

# Sets
# Warehouse = 
model.Warehouses = Set(initialize = ['Madrid', 'Barcelona', 'Malaga', 'Coruna', 'Sevilla'])#RangeSet(1,5)  #Set(initialize)  # Example: 5 potential warehouse locations
model.Customers = Set(initialize = ['Burgos', 'Zaragoza', 'Albacete', 'Murcia','Cordoba','Salamanca', 'Gijon']) #RangeSet(1, 7)  # Example: 4 customers


# Madrid, barcelona, Malaga, Coruna, Sevilla are warehouses
# Destinations: Burgos, Zaragoza, Albacete, Murcia, Cordoba, Salamance, Gijon
# Cost of transporting one good to the location
cost = {
    ('Madrid','Burgos'): 175,
    ('Madrid','Zaragoza'): 275,
    ('Madrid','Albacete'): 220,
    ('Madrid','Murcia'): 340,
    ('Madrid','Cordoba'): 300,
    ('Madrid','Salamanca'): 173,
    ('Madrid','Gijon'): 385,

    ('Barcelona','Burgos'): 490,
    ('Barcelona','Zaragoza'): 258,
    ('Barcelona','Albacete'): 430,
    ('Barcelona','Murcia'): 470,
    ('Barcelona','Cordoba'): 710,
    ('Barcelona','Salamanca'): 652,
    ('Barcelona','Gijon'): 685,

    ('Malaga','Burgos'): 625,
    ('Malaga','Zaragoza'): 625,
    ('Malaga','Albacete'): 335,
    ('Malaga','Murcia'): 320,
    ('Malaga','Cordoba'): 130,
    ('Malaga','Salamanca'): 480,
    ('Malaga','Gijon'): 767,

    ('Coruna','Burgos'): 220,
    ('Coruna','Zaragoza'): 640,
    ('Coruna','Albacete'): 730,
    ('Coruna','Murcia'): 856,
    ('Coruna','Cordoba'): 690,
    ('Coruna','Salamanca'): 350,
    ('Coruna','Gijon'): 220,

    ('Sevilla','Burgos'): 583,
    ('Sevilla','Zaragoza'): 640,
    ('Sevilla','Albacete'): 400,
    ('Sevilla','Murcia'): 430,
    ('Sevilla','Cordoba'): 120,
    ('Sevilla','Salamanca'): 400,
    ('Sevilla','Gijon'): 680
}

# Parameters

# Cost of the warehouse to each of the customer locations
model.Cost = Param(model.Warehouses, model.Customers, initialize=cost)

# Stock in each of the warehouses
stock = {'Madrid': 500, "Barcelona": 500, 'Malaga': 600, 'Coruna': 400, 'Sevilla':330}
model.Stock = Param(model.Warehouses, initialize = stock)


# Demand for each customer
demand = {'Burgos': 50, 'Zaragoza': 30, 'Albacete': 40, 'Murcia': 20, 'Cordoba':80, 'Salamanca':90, 'Gijon':20}
model.Demand = Param(model.Customers, initialize=demand)

# Decision variables: Binary.
model.y = Var(model.Warehouses, within=Binary)


# Objective rule
def objective_rule(model):
    return sum(model.Cost[i, j] * model.y[i] * model.Demand[j] for i in model.Warehouses for j in model.Customers)

# Objective function
model.obj = Objective(rule=objective_rule, sense=minimize)



# Constraints:

# Demand is fulfilled for each customer location
def demand_fulfill(model, j):
    return sum(model.y[i] * model.Demand[j] for i in model.Warehouses) >= model.Demand[j]

model.demand_fulfill = Constraint(model.Customers, rule = demand_fulfill)


# model.demand_fulfillment = ConstraintList()
# for j in model.Customers:
#     model.demand_fulfillment.add(sum(model.y[i] * model.Demand[j] for i in model.Warehouses) >= model.Demand[j])


# Make sure we get the two best warehouse locations
def two_warehouses(model, i):
    return sum(model.y[i] for i in model.Warehouses ) == 2

model.warehouses_two = Constraint(model.Warehouses, rule = two_warehouses)

# Ensure that there is sufficient stock in the Warehouse, to deliver to the customer locations.
def ensure_stock_rule(model, i):
    return sum(model.y[i]*model.Demand[j] for j in model.Customers) <= model.Stock[i]

model.warehouses_stock = Constraint(model.Warehouses, rule=ensure_stock_rule)



# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)


display(model)

Model unknown

  Variables:
    y : Size=5, Index=Warehouses
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
        Barcelona :     0 :   0.0 :     1 : False : False : Binary
           Coruna :     0 :   0.0 :     1 : False : False : Binary
           Madrid :     0 :   1.0 :     1 : False : False : Binary
           Malaga :     0 :   0.0 :     1 : False : False : Binary
          Sevilla :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 212020.0

  Constraints:
    demand_fulfill : Size=7
        Key       : Lower : Body  : Upper
         Albacete :  40.0 :  80.0 :  None
           Burgos :  50.0 : 100.0 :  None
          Cordoba :  80.0 : 160.0 :  None
            Gijon :  20.0 :  40.0 :  None
           Murcia :  20.0 :  40.0 :  None
        Salamanca :  90.0 : 180.0 :  None
         Zaragoza :  30.0 :  60.0 :  None
    warehouses_two : Size=5
        K